In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Falcon Instruct (PEFT)

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_falcon_instruct_peft.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_falcon_instruct_peft.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_falcon_instruct_peft.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a> (A Python-3 GPU notebook is recommended)
  </td>
</table>

## Overview

This notebook demonstrates running inferences locally with prebuilt Falcon Instruct models, deploying prebuilt Falcon Instruct models, finetuning and deploying Falcon Instruct models with performance efficient finetuning libraries ([PEFT](https://github.com/huggingface/peft)), quantizing and deploying Falcon Instruct models with [GPTQ](https://arxiv.org/abs/2210.17323), and evaluating PEFT-finetuned Falcon Instruct models in Vertex AI.

### Objective

- Run inferences locally on prebuilt Falcon Instruct models
- Deploy prebuilt Falcon Instruct models
- Finetune and deploy Falcon Instruct models with PEFT
- Quantize and deploy Falcon Instruct models with GPTQ
- Evaluate PEFT-finetuned Falcon Instruct models

| Models | LoRA |
| :- | :- |
| [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) | Y |
| [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) | Y |

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

Running inferences locally with Falcon Instruct models requires a GPU.

### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    ! pip3 install google-cloud-language==2.10.0
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
! pip3 install transformers==4.31.0
! pip3 install einops==0.6.1
! pip3 install accelerate==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.39.0
    Uninstalling google-cloud-aiplatform-1.39.0:
      Successfully uninstalled google-cloud-aiplatform-1.39.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-language
    Found existing installation: google-cloud-language 2.9.1
    Uninstalling google-cloud-language-2.9.1:
      Successfully uninstalled google-cloud-language-2.9.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 707.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00


### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Compute Engine API, and Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,language.googleapis.com).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

Set the following variables for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the specified region (`REGION`). Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1").

In [ ]:
# Cloud project id.
PROJECT_ID = "<your project id>"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "<region>"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output with gs:// prefix.
BUCKET_URI = "<your cloud storage bucket url>"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "peft")
DATA_BUCKET = os.path.join(EXPERIMENT_BUCKET, "data")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = "<your service account>"  # @param {type:"string"}

Updated property [core/project].


### Initialize Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define constants

In [ ]:
# The pre-built training, serving and evaluation docker images.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231207_0936_RC00"
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-serve:20231129_0948_RC00"
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231127_0916_RC00"
VLLM_GPTQ_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:gptq"
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20231011_0934_RC00"

### Define common functions

In [ ]:
import os
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform, language


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name: str,
    base_model_id: str,
    finetuned_lora_model_path: str,
    service_account: str,
    task: str,
    machine_type: str = "n1-standard-16",
    # accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_type: str = "NVIDIA_TESLA_P100",
    accelerator_count: int = 1,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "BASE_MODEL_ID": base_model_id,
        "TASK": task,
    }
    if finetuned_lora_model_path:
        serving_env["FINETUNED_LORA_MODEL_PATH"] = finetuned_lora_model_path
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=PREDICTION_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/peft_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "n1-standard-16",
    # accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_type: str = "NVIDIA_TESLA_P100",
    accelerator_count: int = 1,
    quantization_method: str = "",
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.9",
        "--disable-log-stats",
        "--dtype=float16",
        "--trust-remote-code",
    ]
    if quantization_method:
        vllm_args.append(f"--quantization={quantization_method}")
    if quantization_method == "gptq":
        vllm_docker_uri = VLLM_GPTQ_DOCKER_URI
    else:
        vllm_docker_uri = VLLM_DOCKER_URI

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=vllm_docker_uri,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint


def moderate_text(text: str) -> language.ModerateTextResponse:
    """Calls Vertex AI APIs to analyze text moderations."""
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.moderate_text(document=document)


def show_text_moderation(text: str, response: language.ModerateTextResponse) -> None:
    """Shows text moderation results."""
    import pandas as pd

    def confidence(category: language.ClassificationCategory) -> float:
        return category.confidence

    columns = ["category", "confidence"]
    categories = sorted(response.moderation_categories, key=confidence, reverse=True)
    data = ((category.name, category.confidence) for category in categories)
    df = pd.DataFrame(columns=columns, data=data)

    print(f"Text analyzed:\n{text}")
    print(df.to_markdown(index=False, tablefmt="presto", floatfmt=".0%"))

## Run inferences locally with prebuilt Falcon Instruct models

You will need at least 16GB of memory to swiftly run inference with Falcon-7B-Instruct.

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
    "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Girafatron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Deploy prebuilt Falcon Instruct models

This section deploys prebuilt Falcon Instruct models on the Endpoint. The model deployment step will take 15 minutes to 40 minutes to complete.

The peak GPU memory usages for [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct), and [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) are ~15.5G and ~84G separately with the default settings. Please adjust the machine type, accelerator type and accelerator count accordingly. We use V100 in deployments as an example. Note that V100 serving generally offers better throughput and latency performance than L4 serving, while L4 serving is generally more cost efficient than V100 serving. The serving efficiency of V100 and L4 GPUs is inferior to that of A100 GPUs, but V100 and L4 GPUs are nevertheless good serving solutions if you do not have A100 quota.

Set the prebuilt model id.

In [ ]:
prebuilt_model_id = "tiiuae/falcon-7b-instruct"  # @param ["tiiuae/falcon-7b-instruct", "tiiuae/falcon-40b-instruct"]

We use the PEFT serving images to deploy prebuilt Falcon Instruct models, by setting finetuning LoRA model paths as empty.

In [ ]:
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute

# Sets V100 (16G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may deploy tiiuae/falcon-40b-instruct with
#  multiple V100s. Please keep in mind that the efficiency of serving with
#  multiple V100s is inferior to that of serving with A100s.
# Compared with L4, V100 serving can have better throughput and latency.
machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
accelerator_type = "NVIDIA_TESLA_P100"
accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 8  # for tiiuae/falcon-40b-instruct

# Sets L4 (24G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may deploy tiiuae/falcon-40b-instruct with
#  multiple L4s. Please keep in mind that the efficiency of serving with
#  multiple L4s is inferior to that of serving with A100s.
# Compared with V100, L4 serving can be more cost efficient.

# For tiiuae/falcon-7b-instruct.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# For tiiuae/falcon-40b-instruct.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Sets A100 (40G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 4  # for tiiuae/falcon-40b-instruct

# Sets A100 (80G) to deploy falcon-40b-instruct models for faster inferences.
# machine_type = "a2-ultragpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 2

model_without_peft, endpoint_without_peft = deploy_model(
    model_name=get_job_name_with_datetime(prefix="falcon-instruct-serve"),
    base_model_id=prebuilt_model_id,
    finetuned_lora_model_path="",  # This will avoid override finetuning models.
    service_account=SERVICE_ACCOUNT,
    task="instruct-lora",
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print("endpoint_name:", endpoint_without_peft.name)

NOTE: The prebuilt model weights will be downloaded on the fly from the original location after the deployment succeeds. Thus, an additional 10-30 minutes of waiting time is needed **after** the above model deployment step succeeds and before you can run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

Example:

```
Human: What is a car?
Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_without_peft.name` allows us to get the
#   endpoint name of the endpoint `endpoint_without_peft` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_without_peft.name
endpoint_name = "<endpoint id>"  # @param {type:"string"}
aip_endpoint_name = (
    f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
)
endpoint_without_peft = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    # {
    #     "prompt": "What is a car?",
    #     "max_tokens": 50,
    #     "temperature": 1.0,
    #     "top_p": 1.0,
    #     "top_k": 10,
    # },
    {
        "prompt": "What is the capital of India",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_without_peft.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
What is the capital of India
Output:
What is the capital of India?
The capital of India is New Delhi, which is located in the northern part of the country.


In [ ]:
response

Prediction(predictions=['Prompt:\nWhat is the capital of India\nOutput:\nWhat is the capital of India?\nThe capital of India is New Delhi, which is located in the northern part of the country.'], deployed_model_id='574549801094348800', model_version_id='1', model_resource_name='projects/698487201632/locations/us-central1/models/1700915900867149824', explanations=None)

## Finetune and deploy Falcon Instruct models with PEFT

This section demonstrates how to finetune and deploy Falcon Instruct models with PEFT LoRA.

Set the base model id.

In [ ]:
base_model_id = "tiiuae/falcon-7b-instruct"  # @param ["tiiuae/falcon-7b-instruct", "tiiuae/falcon-40b-instruct"]

### Finetune

Use the Vertex AI SDK to create and run the custom training jobs with Vertex AI Model Garden training images.

This example uses the dataset [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco). You can either use a [dataset from huggingface](https://huggingface.co/datasets) or a custom JSONL dataset in [Vertex text model dataset format](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format) stored in Cloud Storage. The `template` parameter is optional.

The peak GPU memory usages are ~11G and ~34G for finetuning LoRA models for [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct), and [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) separately with default training parameters and the example dataset. Falcon-7b-instruct can be finetuned on 1 P100/V100, and falcon-40b-instruct can be finetuned on 1 A100 (40G).

#### [Optional] Finetune with a custom dataset

To use a custom dataset, you should supply a `gs://` URI to a JSONL file in [Vertex text model dataset format](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format) in the `dataset_name` below.

For example, here is one data point from the sample dataset `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`:

```json
{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
```

To use this sample dataset that contains `input_text` and `output_text` fields, set `dataset_name` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl` and `template` to `vertex_sample`. For advanced usage with custom datatset fields, see [the template example](https://github.com/tloen/alpaca-lora/blob/main/templates/alpaca.json) and supply your own JSON template as `gs://` URIs.

In [ ]:
BUCKET_URI

NameError: name 'BUCKET_URI' is not defined

In [1]:
from google.cloud import storage

# BUCKET_NAME = "gs://my_bucket"

storage_client = storage.Client()
bucket = storage_client.get_bucket('<your cloud storage bucket>')

# blob = bucket.blob('train.dat')
# blob = blob.download_as_string()
# blob = blob.decode('utf-8')

NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/evolent_data_hitesh?projection=noAcl&prettyPrint=false: The specified bucket does not exist.

In [ ]:
import json
# {
#     "description": "Template used by Alpaca-LoRA.",
#     "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
#     "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
#     "response_split": "### Response:"
# }

def preprocess_data(bucket, dataset):
  blob = bucket.blob(f"{dataset}.dat")
  blob = blob.download_as_string()
  blob = blob.decode('utf-8')
  data_examples = blob.split('\n')
  tab_char = '\t'
  print(type(data_examples))
  label_map = {'1': 'digestive system diseases',
               '2': 'cardiovascular diseases',
               '3': 'neoplasms',
               '4': 'nervous system diseases',
               '5': 'general pathological conditions'}
  final_data = []
  try:
    for example in data_examples:
      final_data.append(str({'input_text': f"You are a medical conversational bot. Classify the given text into an appropriate disease group. The groups are: [digestive system diseases, cardiovascular diseases, neoplasms, nervous system diseases, general pathological conditions] Text: {example.split(tab_char)[1]}", 'output_text': f"{label_map[example.split(tab_char)[0]]}"}))
  except Exception as e:
    pass
  print(len(final_data))
  print(final_data[0])

  # instruction = 'Describe the type of disease'
  # ex_desc = {
  #               "description": "Template used by Alpaca-LoRA.",
  #               "prompt_input": f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
  #               "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
  #               "response_split": "### Response:"
  #           }
  data_jsonl = "\n".join(final_data)

  # with open(f'{dataset}.jsonl', 'w') as f:
  #   f.write(data_jsonl)

  # create a blob
  blob = bucket.blob(f'{dataset}.jsonl')
  # upload the blob
  blob.upload_from_string(
        data=json.dumps(data_jsonl)
        )

  # return data_examples




In [ ]:
preprocess_data(bucket, 'train')

<class 'list'>
14438
{'input_text': 'You are a medical conversational bot. Classify the given text into an appropriate disease group. The groups are: [digestive system diseases, cardiovascular diseases, neoplasms, nervous system diseases, general pathological conditions] Text: Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumfle

In [ ]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
# dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
dataset_name = "<your cloud storage bucket url>/train.jsonl"
# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute

# # Uses V100 (16G) to finetune falcon-7b-instruct.
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# # accelerator_type = "NVIDIA_TESLA_P100"
# accelerator_count = 1

# # # Uses L4 (24G) to finetune falcon-7b-instruct.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# Uses V100 (16G) to finetune falcon-40b-instruct.
# machine_type = "n1-standard-16"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 4

# Uses L4 (24G) to finetune falcon-40b-instruct.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

# Uses A100 (40G) to finetune falcon-40b-instruct.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

replica_count = 1

# Setup training job.
job_name = get_job_name_with_datetime("falcon-instruct-lora-train")
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)
output_dir = os.path.join(MODEL_BUCKET, job_name)
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")

# Pass training arguments and launch job.
max_steps = 10
train_job.run(
    args=[
        "--task=instruct-lora",
        f"--pretrained_model_id={base_model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={output_dir_gcsfuse}",
        "--lora_rank=64",
        "--lora_alpha=16",
        "--lora_dropout=0.1",
        "--warmup_ratio=0.03",
        f"--max_steps={max_steps}",
        "--max_seq_length=512",
        "--learning_rate=2e-4",
        f"--template={template}",
    ],
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("Trained models were saved in: ", output_dir)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://evolent_data_hitesh/temporal/aiplatform-custom-training-2024-02-04-12:48:44.568 


InvalidArgument: 400 GPU Accelerator is required for image "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231207_0936_RC00".

### Deploy
This section uploads the model to Model Registry and deploys it on the Endpoint.

The model deployment step will take 15 minutes to 40 minutes to complete.

The peak GPU memory usages for [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct), and [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) with LoRA weights are ~15.5G and ~84G separately with the default settings. Please adjust the machine type, accelerator type and accelerator count accordingly. We use V100 in deployments as an example. Note that V100 serving generally offers better throughput and latency performance than L4 serving, while L4 serving is generally more cost efficient than V100 serving. The serving efficiency of V100 and L4 GPUs is inferior to that of A100 GPUs, but V100 and L4 GPUs are nevertheless good serving solutions if you do not have A100 quota.

In [ ]:
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute

# Sets V100 (16G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may deploy tiiuae/falcon-40b-instruct with
#  multiple V100s. Please keep in mind that the efficiency of serving with
#  multiple V100s is inferior to that of serving with A100s.
# Compared with L4, V100 serving can have better throughput and latency.
machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
accelerator_type = "NVIDIA_TESLA_P100"
accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 8  # for tiiuae/falcon-40b-instruct

# Sets L4 (24G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may deploy tiiuae/falcon-40b-instruct with
#  multiple L4s. Please keep in mind that the efficiency of serving with
#  multiple L4s is inferior to that of serving with A100s.
# Compared with V100, L4 serving can be more cost efficient.

# For tiiuae/falcon-7b-instruct.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# For tiiuae/falcon-40b-instruct.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Sets A100 (40G) to deploy tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 4  # for tiiuae/falcon-40b-instruct

# Sets A100 (80G) to deploy falcon-40b-instruct models for faster inferences.
# machine_type = "a2-ultragpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100_80GB"
# accelerator_count = 2

model_with_peft, endpoint_with_peft = deploy_model(
    model_name=get_job_name_with_datetime(prefix="falcon-instruct-peft-serve"),
    base_model_id=base_model_id,
    finetuned_lora_model_path=os.path.join(output_dir, "checkpoint-" + str(max_steps)),
    service_account=SERVICE_ACCOUNT,
    task="instruct-lora",
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print("endpoint_name:", endpoint_with_peft.name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/698487201632/locations/us-central1/endpoints/1215156061762224128/operations/1276185485454082048


KeyboardInterrupt: 

NOTE: After the deployment succeeds, the base model weights will be downloaded one the fly from the original location and LoRA model weights will be downloaded from the GCS bucket used in training above. Thus, an additional 10-30 minutes of waiting time is needed **after** the above model deployment step succeeds and before you can run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

Example:

```
Human: What is a car?
Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_with_peft.name` allows us to get the
#   endpoint name of the endpoint `endpoint_with_peft` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_with_peft.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint_with_peft = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_with_peft.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

## Quantize and deploy Falcon Instruct models

This section demonstrates post-training quantization of Falcon Instruct models with Vertex Custom Job. Quantization reduces the memory required by a model while attempting to retain the same performance. Read more about GPTQ in the following publication: [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323).

### Deploy pre-quantized models with Google Cloud Text Moderation
Many GPTQ-quantized models are provided [here](https://huggingface.co/TheBloke?search_models=-gptq).

This section uploads the model to Model Registry and deploys it on the Endpoint.

The model deployment step will take 15 minutes to 1 hour to complete, depending on the model sizes.

Notice that deploying a quantized requires much less GPU. We can deploy a quantized 40B model with only two L4s instead of four.

In [ ]:
quantized_model_id = "TheBloke/Falcon-7B-Instruct-GPTQ"  # @param ["TheBloke/Falcon-7B-Instruct-GPTQ", "TheBloke/falcon-40b-instruct-GPTQ"]

quantization_method = "gptq"

# Finds Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 1 L4 (24G) to deploy Falcon Instruct 7B model.
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Sets 2 L4's (24G) to deploy Falcon Instruct 40B model.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

model_prequantized_vllm, endpoint_prequantized_vllm = deploy_model_vllm(
    model_name=get_job_name_with_datetime(
        prefix="falcon-instruct-serve-vllm-prequantized"
    ),
    model_id=quantized_model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    quantization_method=quantization_method,
)

NOTE: After the deployment succeeds, the model weights will be downloaded on the fly. Thus additional 10 ~ 40 minutes (depending on the model sizes) of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

Example:

```
Human: What is a car?
Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_prequantized_vllm.name` allows us to get the
#   endpoint name of the endpoint `endpoint_prequantized_vllm` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_prequantized_vllm.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint_prequantized_vllm = aiplatform.Endpoint(aip_endpoint_name)


# Overides max_length and top_k parameters during inferences.
# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the max length, such as set max_length as 20.
# If you are using L4 GPUs to serve Falcon Instruct 40B models, you should set
# max_length to around 1,000 tokens or fewer. If you need longer generated
# sequences, please file a request with Vertex to allowlist your project for a
# longer timeout threshold with Vertex endpoints.
instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_prequantized_vllm.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Text moderation analyzes a document against a list of safety attributes, which include "harmful categories" and topics that may be considered sensitive.

In [ ]:
for generated_text in response.predictions:
    # Send a request to the API.
    response = moderate_text(generated_text)
    # Show the results.
    show_text_moderation(generated_text, response)

### Quantize Falcon Instruct models

Quantization reduces the amount of GPU required to serve a model by reducing the bit precision of the weights while minimizing drop in performance. Serving quantized models on VLLM requires models to be quantized to 4 bits. It is recommended to first search if a model has already been quantized and made publicly available: [GPTQ](https://huggingface.co/TheBloke?search_models=-gptq).

Quantizing models with GPTQ will take around 40 minutes for Falcon Instruct 7B using 1 NVIDIA_L4 GPU and will take around 4 hours for Falcon Instruct 40B using 4 NVIDIA_L4 GPU.

Finetuned models can also be quantized, so long as the LoRA weights are merged with the base model.

Set the base model id.

In [ ]:
base_model_id = "tiiuae/falcon-7b-instruct"  # @param ["tiiuae/falcon-7b-instruct", "tiiuae/falcon-40b-instruct"]

In [ ]:
# Set up quantization job.

# Set `finetuned_model_path` to the GCS path of the merged finetuned model
# from the section above, if not set, the base model will be quantized.
finetuned_model_path = ""  # @param {type:"string"}
if finetuned_model_path:
    prequantized_model_path = finetuned_model_path
else:
    prequantized_model_path = base_model_id

quantization_method = "gptq"
quantization_job_name = get_job_name_with_datetime(
    f"falcon-instruct-{quantization_method}-quantize"
)

quantization_output_dir = os.path.join(MODEL_BUCKET, quantization_job_name)
quantization_output_dir_gcsfuse = quantization_output_dir.replace("gs://", "/gcs/")

# Worker pool spec.

# Set 1 L4 (24G) for quantizing 7b model.
machine_type = "g2-standard-32"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Set 4 L4 (24G) for quantizing 40b model.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Quantization parameters.
quantization_precision_mode = "4bit"

# The original datasets used in GPTQ paper.
gptq_dataset_name = "c4"  # @param ["wikitext2","c4","c4-new","ptb","ptb-new"]
group_size = -1
damp_percent = 0.1
desc_act = True
quantization_args = [
    "--task=quantize-model",
    f"--quantization_method={quantization_method}",
    f"--pretrained_model_id={base_model_id}",
    f"--quantization_precision_mode={quantization_precision_mode}",
    f"--quantization_output_dir={quantization_output_dir_gcsfuse}",
    f"--quantization_dataset_name={gptq_dataset_name}",
    f"--group_size={group_size}",
    f"--damp_percent={damp_percent}",
    f"--desc_act={desc_act}",
    "--cache_examples_on_gpu=False",
]

# Pass quantization arguments and launch job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": 1,
        "disk_spec": {
            "boot_disk_type": "pd-ssd",
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": TRAIN_DOCKER_URI,
            "env": [
                {
                    "name": "PYTORCH_CUDA_ALLOC_CONF",
                    "value": "max_split_size_mb:32",
                },
            ],
            "command": [],
            "args": quantization_args,
        },
    }
]

print(f"Quantizing {prequantized_model_path}.")
quantize_job = aiplatform.CustomJob(
    display_name=quantization_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)
quantize_job.run()

print("Quantized models were saved in: ", quantization_output_dir)

### Deploy quantized models with Google Cloud Text Moderation
This section uploads the model to Model Registry and deploys it on the Endpoint.

The model deployment step will take 15 minutes to 1 hour to complete, depending on the model sizes.

Notice that deploying a quantized model requires much less GPU. We can deploy a quantized 40B model with only two L4 GPUs instead of four.

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets 1 L4 (24G) to deploy Falcon Instruct 7B.
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Sets 2 L4's (24G) to deploy Falcon Instruct 70B models.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

model_quantized_vllm, endpoint_quantized_vllm = deploy_model_vllm(
    model_name=get_job_name_with_datetime(
        prefix="falcon-instruct-serve-vllm-quantized"
    ),
    model_id=quantization_output_dir,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    quantization_method=quantization_method,
)

NOTE: After the deployment succeeds, the model weights will be downloaded on the fly. Thus additional 10 ~ 40 minutes (depending on the model sizes) of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

Example:

```
Human: What is a car?
Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_quantized_vllm.name` allows us to get the
#   endpoint name of the endpoint `endpoint_quantized_vllm` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_quantized_vllm.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint_quantized_vllm = aiplatform.Endpoint(aip_endpoint_name)


# Overides max_length and top_k parameters during inferences.
# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the max length, such as set max_length as 20.
# If you are using L4 GPUs to serve Falcon Instruct 40B models, you should set
# max_length to around 1,000 tokens or fewer. If you need longer generated
# sequences, please file a request with Vertex to allowlist your project for a
# longer timeout threshold with Vertex endpoints.
instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_quantized_vllm.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Text moderation analyzes a document against a list of safety attributes, which include "harmful categories" and topics that may be considered sensitive.

In [ ]:
for generated_text in response.predictions:
    # Send a request to the API.
    response = moderate_text(generated_text)
    # Show the results.
    show_text_moderation(generated_text, response)

## Evaluate PEFT-finetuned Falcon Instruct models

This section demonstrates how to evaluate the Falcon Instruct models fintuned with PEFT LoRA using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex CustomJob. Please reference the peak GPU memory usgaes for serving and adjust the machine type, accelerator type and accelerator count accordingly.

This example uses the dataset [TruthfulQA](https://arxiv.org/abs/2109.07958). All supported tasks are listed in [this task table](https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md).

In [ ]:
eval_dataset = "truthfulqa_mc"  # @param {type:"string"}

# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute

# Sets V100 (16G) to evaluate tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may evaluate tiiuae/falcon-40b-instruct with
#  multiple V100s. Please keep in mind that the efficiency of evaluating with
#  multiple V100s is inferior to that of evaluating with A100s.
# Compared with L4, V100 inference can have better throughput and latency.
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 8  # for tiiuae/falcon-40b-instruct

# Sets L4 (24G) to evaluate tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# If A100 is not available, you may evaluate tiiuae/falcon-40b-instruct with
#  multiple L4s. Please keep in mind that the efficiency of evaluating with
#  multiple L4s is inferior to that of evaluating with A100s.
# Compared with V100, L4 inference can be more cost efficient.

# For tiiuae/falcon-7b-instruct.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# For tiiuae/falcon-40b-instruct.
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

# Sets A100 (40G) to evaluate tiiuae/falcon-7b-instruct or tiiuae/falcon-40b-instruct.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1  # for tiiuae/falcon-7b-instruct
# accelerator_count = 4  # for tiiuae/falcon-40b-instruct

# Sets A100 (80G) to evaluate falcon-40b-instruct models for faster inferences.
# machine_type = "a2-ultragpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100_80GB"
# accelerator_count = 2

replica_count = 1

# Setup evaluation job.
job_name = get_job_name_with_datetime(prefix="falcon-instruct-peft-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")

In [ ]:
# Prepare evaluation command that runs the evaluation harness.
# Set `trust_remote_code = True` because evaluating the model requires
# executing code from the model repository.
# Set `use_accelerate = True` to enable evaluation across multiple GPUs.
eval_command = [
    "python",
    "main.py",
    "--model",
    "hf-causal-experimental",
    "--model_args",
    f"pretrained={base_model_id},peft={output_dir_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    "--tasks",
    f"{eval_dataset}",
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
]

### Submit evaluation CustomJob

In [ ]:
# Pass evaluation arguments and launch job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "command": eval_command,
            "args": [],
        },
    }
]

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
)

eval_job.run()

print("Evaluation results were saved in:", eval_output_dir)

### Fetch and print evaluation results

In [ ]:
import json

from google.cloud import storage

# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = BUCKET_URI.split("gs://")[1]
bucket = storage_client.get_bucket(BUCKET_NAME)
RESULT_FILE_PATH = eval_output_dir[len(BUCKET_URI) + 1 :]
blob = bucket.blob(RESULT_FILE_PATH)
raw_result = blob.download_as_string()

# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

## Clean up resources

In [ ]:
# Delete custom train, quantization, and evaluation jobs.
train_job.delete()
quantize_job.delete()
eval_job.delete()

# Undeploy models and delete endpoints.
endpoint_without_peft.delete(force=True)
endpoint_with_peft.delete(force=True)
endpoint_prequantized_vllm.delete(force=True)
endpoint_quantized_vllm.delete(force=True)

# Delete models.
model_without_peft.delete()
model_with_peft.delete()
model_prequantized_vllm.delete()
model_quantized_vllm.delete()